# Fonctions de hachage

On rappelle qu'une fonction de hachage est une fonction qui à toute clé d'un ensemble $C$ associe un élément d'un ensemble $N$ fini de cardinal raisonnable.

#### Question 1
Le *hachage par division* de largeur $m$ est une fonction de hachage définie sur les entiers et à valeurs dans $⟦ 0, m-1 ⟧$, qui à tout entier $k$ associe le reste de la division euclidienne de $k$ par $m$.<br/>

Écrire une fonction `hash_int : int -> int -> int` qui prend en argument deux entiers $m$ et $k$ et qui renvoie l'image de $k$ pour la fonction de hachage par division de largeur $m$ de $k$.

In [1]:
let hash_int m k =
  k mod m
;;

val hash_int : int -> int -> int = <fun>


#### Question 2
Dans le cas d'une chaîne de caractères, on va d'abord coder la chaîne sous forme d'entier avant de lui appliquer la fonction de hachage : la chaîne $s_0\dots s_{n-1}$ pourra être codée par l'entier $\displaystyle \sum_{k=0}^{n-1} a_k 256^k$ où $a_k$ est le code ASCII du caractère $s_k$.<br/>

Écrire une fonction `int_of_str : string -> int` qui prend en argument une chaîne de caractères et qui renvoie l'entier décrit ci-dessus.

In [2]:
let int_of_string str =
  let n = String.length str in
  let somme = ref 0 in
  for i = n-1 downto 0 do
    somme := 256 * !somme + int_of_char (String.get str i)
  done;
  !somme
;;

val int_of_string : string -> int = <fun>


#### Question 3
En déduire une fonction `hash_string : int -> string -> int` de hachage pour les chaînes de caractères.

In [3]:
(* Version naïve, en utilisant les deux fonctions précédentes *)
let hash_string m str =
  let code = int_of_string str in
  hash_int m code
;;

(* Pour éviter les problèmes de débordement *)
let hash_string m str =
  let n = String.length str in
  let somme = ref 0 in
  for i = n-1 downto 0 do
    somme := (256 * !somme + int_of_char (String.get str i)) mod m
  done;
  !somme
;;

val hash_string : int -> string -> int = <fun>


val hash_string : int -> string -> int = <fun>


# Tables de hachage de taille fixe

On représente une table de hachage par le type

In [4]:
type ('a, 'b) table = {
    h : 'a -> int ;
    contenu : ('a * 'b) list array
  };;

type ('a, 'b) table = { h : 'a -> int; contenu : ('a * 'b) list array; }


#### Question 4
Écrire une fonction `creer_table : (’a -> int) -> int -> (’a, ’b) table` prenant en argument une fonction de hachage et la largeur de la table et qui renvoie une table de hachage vide.

In [5]:
let creer_table fonction_h largeur = {
  h = fonction_h ;
  contenu = Array.make largeur []
}

val creer_table : ('a -> int) -> int -> ('a, 'b) table = <fun>


#### Question5
Écrire une fonction `contient : ('a, 'b) table -> 'a -> bool` qui prend en argument une table et une clé et qui renvoie `true` si la table contient la clé, `false` sinon.

In [6]:
let rec liste_contient lst cle =
  match lst with
  | [] -> false
  | (k, _)::q -> k = cle || liste_contient q cle
;;
let contient tbl cle =
  let i = tbl.h cle in
  liste_contient tbl.contenu.(i) cle
;;

val liste_contient : ('a * 'b) list -> 'a -> bool = <fun>


val contient : ('a, 'b) table -> 'a -> bool = <fun>


#### Question 6
Écrire une fonction `trouver : ('a, 'b) table -> 'a -> 'b` qui prend en argument une table et une clé et qui renvoie la valeur associée à la clé si elle existe.

In [7]:
let rec liste_trouve lst cle =
  match lst with
  | [] -> raise Not_found
  | (k, v)::q ->
     if k = cle
     then v
     else liste_trouve q cle
;;

let trouver tbl cle =
  let i = tbl.h cle in
  liste_trouve tbl.contenu.(i) cle
;;

val liste_trouve : ('a * 'b) list -> 'a -> 'b = <fun>


val trouver : ('a, 'b) table -> 'a -> 'b = <fun>


#### Question 7
Écrire une fonction `ajouter : ('a, 'b) table -> 'a -> 'b -> unit` qui prend en argument une table, une clé $c$ et une valeur $v$ et qui ajoute le couple $(c,v)$. On précisera le comportement de la fonction si la clé est déjà présente dans la table.

In [8]:
(* Ici, on écrase l'ancienne valeur si la clé est déjà dans la table *)
let rec ajout_dans_liste lst cle valeur =
  match lst with
  | [] -> [(cle, valeur)]
  | (k, v)::q ->
     if k = cle
     then (cle, valeur)::q
     else (k, v)::ajout_dans_liste q cle valeur
;;
let ajouter tbl cle valeur =
  let i = tbl.h cle in
  let new_liste =
    ajout_dans_liste tbl.contenu.(i) cle valeur
  in tbl.contenu.(i) <- new_liste
;;

val ajout_dans_liste : ('a * 'b) list -> 'a -> 'b -> ('a * 'b) list = <fun>


val ajouter : ('a, 'b) table -> 'a -> 'b -> unit = <fun>


#### Question 8
Écrire une fonction `supprimer : ('a, 'b) table -> 'a -> unit` qui prend en argument une table et une clé $c$ et qui supprime l'entrée associée à la clé $c$ dans la table. 

In [9]:
let rec suppr_dans_liste lst cle =
  match lst with
  | [] -> []
  | (k, v)::q ->
     if k = cle
     then q
     else (k, v)::suppr_dans_liste q cle
;;
let supprimer tbl cle =
  let i = tbl.h cle in
  let new_liste =
    suppr_dans_liste tbl.contenu.(i) cle
  in tbl.contenu.(i) <- new_liste
;;

val suppr_dans_liste : ('a * 'b) list -> 'a -> ('a * 'b) list = <fun>


val supprimer : ('a, 'b) table -> 'a -> unit = <fun>


# Tables de hachage de taille dynamique

## Une première approche
Lorsque le nombre de collisions est important, la taille des les listes contenues dans le tableau augmente. Cela sera notamment le cas si on ajoute de nombreuses entrées dans la table.<br/>

Or la partie coûteuse de la recherche d'une clé est le parcours de la liste correspondant à l'image de la clé par la fonction de hachage.<br/>


On se propose d'améliorer ce point en utilisant des tables de hachage de taille dynamique : lorsque le nombre d'éléments devient trop important, on augmente la taille du tableau (ce qui impose de rajouter un paramètre à la fonction de hachage).

On utilisera donc le type 

In [10]:
type ('a, 'b) table_dyn = {
    h : int -> 'a -> int ;
    mutable nb : int ;
    mutable contenu : ('a * 'b) list array
  };;

type ('a, 'b) table_dyn = {
  h : int -> 'a -> int;
  mutable nb : int;
  mutable contenu : ('a * 'b) list array;
}


où `nb` est le nombre d'éléments présents dans la table.

#### Question 9
Écrire une fonction `creer_table_dyn : (int -> ’a -> int) -> int -> (’a, ’b) table_dyn` prenant en argument une fonction de hachage et la largeur initiale de la table et qui renvoie une table de hachage vide.

In [11]:
let creer_table_dyn fonction_h largeur = {
  h = fonction_h ;
  nb = 0;
  contenu = Array.make largeur []
}

val creer_table_dyn : (int -> 'a -> int) -> int -> ('a, 'b) table_dyn = <fun>


#### Question 10
Écrire une fonction `contient_dyn : ('a, 'b) table_dyn -> 'a -> bool` qui prend en argument une table et une clé et qui renvoie `true` si la table contient la clé, `false` sinon.

In [12]:
let contient_dyn tbl cle =
  let i = tbl.h (Array.length tbl.contenu) cle in
  liste_contient tbl.contenu.(i) cle
;;

val contient_dyn : ('a, 'b) table_dyn -> 'a -> bool = <fun>


#### Question 11
Écrire une fonction `trouver_dyn : ('a, 'b) table_dyn -> 'a -> 'b` qui prend en argument une table et une clé et qui renvoie la valeur associée à la clé si elle existe.

In [13]:
let trouver_dyn tbl cle =
  let i = tbl.h (Array.length tbl.contenu) cle in
  liste_trouve tbl.contenu.(i) cle
;;

val trouver_dyn : ('a, 'b) table_dyn -> 'a -> 'b = <fun>


#### Question 12
Si le nombre d'éléments de la table dépasse le double de la largeur de la table, alors on double la largeur de la table. Écrire une fonction `redimensionne_dyn : ('a, 'b) table_dyn -> unit` qui réarrange la table passée en argument en doublant sa largeur.

In [14]:
let redimensionne_dyn tbl =
  let new_largeur = 2 * Array.length tbl.contenu in
  let new_contenu = Array.make new_largeur [] in
  let rec ajoute_tous liste =
    match liste with
    | [] -> ()
    | (k, v)::q -> 
      let i = tbl.h new_largeur k in
      new_contenu.(i) <- ajout_dans_liste new_contenu.(i) k v ;
      ajoute_tous q
  in
  for i = 0 to Array.length tbl.contenu - 1 do
    ajoute_tous tbl.contenu.(i)
  done;
  tbl.contenu <- new_contenu
;;

val redimensionne_dyn : ('a, 'b) table_dyn -> unit = <fun>


#### Question 13
En déduire une fonction `ajouter_dyn : ('a, 'b) table_dyn -> 'a -> 'b -> unit` qui prend en argument une table, une clé $c$ et une valeur $v$ et qui ajoute le couple $(c,v)$ en redimensionnant la table si nécessaire.

In [15]:
(* On modifie ajout_dans_liste pour renvoyer aussi un booléen indiquant si la longueur de la liste a augmenté*)
let rec ajout_dans_liste_dyn lst cle valeur =
  match lst with
  | [] -> [(cle, valeur)], true
  | (k, v)::q ->
     if k = cle
     then (cle, valeur)::q, false
     else 
       let new_q, ajout = ajout_dans_liste_dyn q cle valeur
       in (k,v)::new_q, ajout       
;;

let ajouter_dyn tbl cle valeur =
  let i = tbl.h (Array.length tbl.contenu) cle in
  let new_liste, ajout =
    ajout_dans_liste_dyn tbl.contenu.(i) cle valeur
  in tbl.contenu.(i) <- new_liste ;
  if ajout
  then 
    begin      
      tbl.nb <- tbl.nb + 1;
      if tbl.nb > 2 * Array.length tbl.contenu
      then redimensionne_dyn tbl
    end
;;

val ajout_dans_liste_dyn :
  ('a * 'b) list -> 'a -> 'b -> ('a * 'b) list * bool = <fun>


val ajouter_dyn : ('a, 'b) table_dyn -> 'a -> 'b -> unit = <fun>


#### Question 14
Écrire une fonction `supprimer_dyn : ('a, 'b) table_dyn -> 'a -> unit` qui prend en argument une table et une clé $c$ et qui supprime l'entrée associée à la clé $c$ dans la table. 

In [16]:
(* On ajoute un booléen dans la fonction suppr_dans_liste pour indiquer si un élément a été supprimé *)
let rec suppr_dans_liste_dyn lst cle =
  match lst with
  | [] -> [], false
  | (k, v)::q ->
     if k = cle
     then q, true
     else 
       let new_q, suppr = suppr_dans_liste_dyn q cle
       in (k, v)::new_q, suppr
;;
let supprimer_dyn tbl cle =
  let i = tbl.h (Array.length tbl.contenu) cle in
  let new_liste, suppr =
    suppr_dans_liste_dyn tbl.contenu.(i) cle
  in 
  if suppr
  then
    begin
      tbl.contenu.(i) <- new_liste ;
      tbl.nb <- tbl.nb - 1
    end
;;

val suppr_dans_liste_dyn : ('a * 'b) list -> 'a -> ('a * 'b) list * bool =
  <fun>


val supprimer_dyn : ('a, 'b) table_dyn -> 'a -> unit = <fun>


## Largeur et nombre premier
Les tables de hachage sont réputées  mieux fonctionner si la taille du tableau utilisé est un
nombre premier. 

On va donc construire un tableau de nombres premiers proches de puissances de 2. Pour cela, on utilisera le tableau `t` de longueur $N=40$ défini ci-dessous, qui vérifie : pour tout $i \in ⟦ 0, N ⟦$, $2^{i}-t.(i)$ est un nombre premier.

In [17]:
(* tables des nombres premiers proches d'une puissance de deux :
   si t.(i) vaut k, alors 2**i - k est premier.
   Les valeurs de t sont toutes strictement positives, sauf la première, négative.
   t est de longueur 40.
*)

let t =
  [| -1; (* 2**0 - (-1) est premier *)
     0; (* 2**1 - 0 est premier *)
     1; (* 2**2 - 1 est premier *)
     1; (* 2**3 - 1 est premier *)
     3; (* 2**4 - 3 est premier *)
     1;
     3;
     1;
     5;
     3;
     3;
     9;
     3;
     1;
     3;
     19;
     15;
     1;
     5;
     1;
     3;
     9;
     3;
     15;
     3;
     39;
     5;
     39;
     57;
     3;
     35;
     1;
     5;
     9;
     41;
     31;
     5;
     25;
     45;
     7; (* 2**39 - 7 est premier *)
    |]

val t : int array =
  [|-1; 0; 1; 1; 3; 1; 3; 1; 5; 3; 3; 9; 3; 1; 3; 19; 15; 1; 5; 1; 3; 9; 3;
    15; 3; 39; 5; 39; 57; 3; 35; 1; 5; 9; 41; 31; 5; 25; 45; 7|]


#### Question 15
Construire un tableau `premiers` tel que pour tout $i \in ⟦ 0, N ⟦$, `premiers.(i)` contient $2^{i}-t.(i)$.

In [18]:
let premiers = 
  let n = Array.length t in
  let tab = Array.make n 0 in
  let puiss2 = ref 1 in
  for i = 0 to n-1 do
    tab.(i) <- !puiss2 - t.(i);
    puiss2 := 2 * !puiss2
  done;
  tab
;;

val premiers : int array =
  [|2; 2; 3; 7; 13; 31; 61; 127; 251; 509; 1021; 2039; 4093; 8191; 16381;
    32749; 65521; 131071; 262139; 524287; 1048573; 2097143; 4194301; 8388593;
    16777213; 33554393; 67108859; 134217689; 268435399; 536870909;
    1073741789; 2147483647; 4294967291; 8589934583; 17179869143; 34359738337;
    68719476731; 137438953447; 274877906899; 549755813881|]


#### Question 16
Reprendre les questions de la partie précédente de sorte que les largeurs de tables appartiennent à la liste `premiers`. On pourra modifier le type `table_dyn` pour y ajouter toute donnée pertinente.

In [19]:
type ('a, 'b) table_dyn = {
    h : int -> 'a -> int ;
    mutable nb : int ;
    mutable i : int ; (* contient l'indice correspondant à la longueur du contenu dans le tableau premiers *)
    mutable contenu : ('a * 'b) list array
  };;
  
let creer_table_dyn fonction_h = {
  h = fonction_h ;
  nb = 0;
  i = 0;
  contenu = Array.make premiers.(0) []
}

(* Les fonctions contient_dyn, trouver_dyn, ajouter_dyn et supprimer_dyn peuvent être reprises sans modification *)

let redimensionne_dyn tbl =
  tbl.i <- tbl.i + 1;
  let new_contenu = Array.make premiers.(tbl.i) [] in
  let rec ajoute_tous liste =
    match liste with
    | [] -> ()
    | (k, v)::q -> 
      let i = tbl.h premiers.(tbl.i) k in
      new_contenu.(i) <- ajout_dans_liste new_contenu.(i) k v ;
      ajoute_tous q
  in
  for i = 0 to Array.length tbl.contenu - 1 do
    ajoute_tous tbl.contenu.(i)
  done;
  tbl.contenu <- new_contenu
;;

type ('a, 'b) table_dyn = {
  h : int -> 'a -> int;
  mutable nb : int;
  mutable i : int;
  mutable contenu : ('a * 'b) list array;
}


val creer_table_dyn : (int -> 'a -> int) -> ('a, 'b) table_dyn = <fun>


val redimensionne_dyn : ('a, 'b) table_dyn -> unit = <fun>
